## Profilers for Python

In this session we will cover two kinds of profiling: deterministic and statistical profiling. 

We'll use both simpler approaches to profile software and more involved softwares.

### Built-in Jupyter methods

In [1]:
def foo():
    aux = 0
    for i in range(10000):
        aux = i * i

In [2]:
%timeit foo()

521 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%prun foo()

### Tracing

You can set a trace function that's triggered every time the VM enters or exits both Python and C functions.

The python interpreter converts programs to bytecodes before beginning execution. Execution itself consist of looping over these bytecodes and performing specific operations over each one.

In [5]:
# ⚠️ Run this code on your Python CLI

def fib(n):
    i, f1, f2 = 1, 1, 1
    while i < n:
        f1, f2 = f2, f1 + f2
        i += 1
    return f1

import opcode


def show_trace(frame, event, arg):
    frame.f_trace_opcodes = True
    code = frame.f_code
    offset = frame.f_lasti

    print(f"| {event:10} | {str(arg):>4} |", end=' ')
    print(f"{frame.f_lineno:>4} | {frame.f_lasti:>6} |", end=' ')
    print(f"{opcode.opname[code.co_code[offset]]:<18} | {str(frame.f_locals):<35} |")
    return show_trace

import sys

header = f"| {'event':10} | {'arg':>4} | line | offset | {'opcode':^18} | {'locals':^35} |"
print(header)
sys.settrace(show_trace)
fib(3)
sys.settrace(None)

| event      |  arg | line | offset |       opcode       |               locals                |
| call       | None |  566 |     -1 | <0>                | {'self': <traitlets.traitlets.Bool object at 0x7ff3d36a0610>, 'obj': <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7ff3d51207c0>, 'cls': <class 'ipykernel.zmqshell.ZMQInteractiveShell'>} |
| line       | None |  574 |      0 | LOAD_FAST          | {'self': <traitlets.traitlets.Bool object at 0x7ff3d36a0610>, 'obj': <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7ff3d51207c0>, 'cls': <class 'ipykernel.zmqshell.ZMQInteractiveShell'>} |
| opcode     | None |  574 |      0 | LOAD_FAST          | {'self': <traitlets.traitlets.Bool object at 0x7ff3d36a0610>, 'obj': <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7ff3d51207c0>, 'cls': <class 'ipykernel.zmqshell.ZMQInteractiveShell'>} |
| opcode     | None |  574 |      2 | LOAD_CONST         | {'self': <traitlets.traitlets.Bool object at 0x7ff3d36a0610>, 'obj': <ipykernel.zm

| opcode     | None |    5 |     16 | POP_JUMP_IF_FALSE  | {'n': 3, 'i': 3, 'f1': 2, 'f2': 3}  |
| line       | None |    8 |     42 | LOAD_FAST          | {'n': 3, 'i': 3, 'f1': 2, 'f2': 3}  |
| opcode     | None |    8 |     42 | LOAD_FAST          | {'n': 3, 'i': 3, 'f1': 2, 'f2': 3}  |
| opcode     | None |    8 |     44 | RETURN_VALUE       | {'n': 3, 'i': 3, 'f1': 2, 'f2': 3}  |
| return     |    2 |    8 |     44 | RETURN_VALUE       | {'n': 3, 'i': 3, 'f1': 2, 'f2': 3}  |
| opcode     | None |   28 |      6 | POP_TOP            | {'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def foo():\n    aux = 0\n    for i in range(10000):\n        aux = i * i', "get_ipython().run_line_magic('timeit', 'foo()')", "get_ipython().run_line_magic('prun', 'foo()')", "get_ipyth

| opcode     | None |  477 |    156 | LOAD_FAST          | {'code': <code object <module> at 0x7ff3d4fbd710, file "/var/folders/82/q80804mx20q_xk4c7mffbbs40000gn/T/ipykernel_63229/4067019739.py", line 29>, 'byte_increments': b'', 'line_increments': b'', 'bytecode_len': 14, 'lastlineno': None, 'lineno': 29, 'addr': 0} |
| opcode     | None |  477 |    158 | LOAD_FAST          | {'code': <code object <module> at 0x7ff3d4fbd710, file "/var/folders/82/q80804mx20q_xk4c7mffbbs40000gn/T/ipykernel_63229/4067019739.py", line 29>, 'byte_increments': b'', 'line_increments': b'', 'bytecode_len': 14, 'lastlineno': None, 'lineno': 29, 'addr': 0} |
| opcode     | None |  477 |    160 | BUILD_TUPLE        | {'code': <code object <module> at 0x7ff3d4fbd710, file "/var/folders/82/q80804mx20q_xk4c7mffbbs40000gn/T/ipykernel_63229/4067019739.py", line 29>, 'byte_increments': b'', 'line_increments': b'', 'bytecode_len': 14, 'lastlineno': None, 'lineno': 29, 'addr': 0} |
| opcode     | None |  477 |    162 |

### Deterministic profiling

#### cProfile

- Offers you both the total running time of cProfile.run(statement, filename=None, sort=-1)a software as well as specific function calls and times
- Nice integration with GUI tools and pstats

In [6]:
import cProfile

# cProfile.run(statement, filename=None, sort=-1)

You can pass python code or a function name that you want to profile as a string to the statement argument.

In [7]:
import numpy as np

cProfile.run("2**200000")

         3 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- ncalls : Shows the number of calls made
- tottime: Total time taken by the given function. Note that the time made in calls to sub-functions are excluded.
- percall: Total time / No of calls. ( remainder is left out )
- cumtime: Unlike tottime, this includes time spent in this and all subfunctions that the higher-level function calls. It is most useful and is accurate for recursive functions.
- The percall following cumtime is calculated as the quotient of cumtime divided by primitive calls. The primitive calls include all the calls that were not included through recursion.

In [8]:
def add_emoji():
    arr=[]
    arr.append('🔥')

def multiply():
    arr=[]
    for i in range(0,400000):
        arr.append(i * 2)
        add_emoji()

def main():
    multiply()
    print('end')

if __name__ == '__main__':
    profiler = cProfile.run('main()')

end
         1200042 function calls in 0.946 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   400000    0.214    0.000    0.402    0.000 4005984123.py:1(add_emoji)
        1    0.007    0.007    0.946    0.946 4005984123.py:11(main)
        1    0.425    0.425    0.936    0.936 4005984123.py:5(multiply)
        1    0.000    0.000    0.946    0.946 <string>:1(<module>)
        3    0.000    0.000    0.003    0.001 iostream.py:208(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:420(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:439(_schedule_flush)
        2    0.000    0.000    0.003    0.002 iostream.py:502(write)
        3    0.000    0.000    0.000    0.000 iostream.py:97(_event_pipe)
        3    0.003    0.001    0.003    0.001 socket.py:480(send)
        3    0.000    0.000    0.000    0.000 threading.py:1059(_wait_for_tstate_lock)
        3    0.000    0.000    0.0

You can save the data using the following:

In [9]:
import  pstats
# p = pstats.Stats('profile.dat', stream = s)
stats = pstats.Stats(profiler)
stats.dump_stats('data.txt')

And use a GUI to visualize it called snakeviz:

In [14]:
# installing the module
!pip install snakeviz

In [15]:
%load_ext snakeviz

In [16]:
import glob

In [17]:
# load it on the notebook
%snakeviz glob.glob('*.txt')

 
*** Profile stats marshalled to file '/var/folders/82/q80804mx20q_xk4c7mffbbs40000gn/T/tmpeeeaj5kz'.
Embedding SnakeViz in this document...


In [18]:
# opens snakeviz
%snakeviz main()

end
 
*** Profile stats marshalled to file '/var/folders/82/q80804mx20q_xk4c7mffbbs40000gn/T/tmp316kdwd_'.
Embedding SnakeViz in this document...


cProfile has a lot more to offer and I recommend checking the [Python docs](https://docs.python.org/3/library/profile.html#module-cProfile) to learn more about its specific functions.

### Statistical profiling

Samples the program counter at regular intervals. The numbers will be statistical approximations instead of exact numbers because of the several process in place.

- Less data to analyze
- Smaller profiling footprint

#### pprofile

pprofile offers both deterministic and statistical modes for profiling. We're going to take a look in the statistical mode:

In your CLI, after installing `pprofile` run it in the statistical mode:

```
pprofile --statistic 0.01 --exclude-syspath test.py
```

In [20]:
# ⚠️ Run this code on your Python CLI

import threading
import time


def func():
    time.sleep(1)

def func2():
    pass

t1 = threading.Thread(target=func)
t2 = threading.Thread(target=func)
t1.start()
t2.start()
(func(), func2())
t1.join()
t2.join()

Hit: indicates the number of calls to each function during the run